##### Implementing Naive Bayes Classifier from scratch

## Background
- Problem type: classification
- Assumptions on data: Independence between the features
- Theory: Bayes' theorem is mathematical formula used for calculating conditional probabilities.
    $ P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)} $
    - A and B are events and $P(B)\neq 0$.
    - P(A), P(B) are called the marginal probability, and represents the probability of an event irrespective of the outcomes of other random variables.
    - $P(A|B)$ is a conditional probability: the likelihood of event A occurring given that B occurred.
    A and B must be different events.
    - and for n different classes, and y as a data point that we would like to assign a class:
 $P(y|x_1, x_2,...,x_n) = \frac{P(x_1, x_2,...,x_n|y) \cdot P(y)}{P(x_1, x_2,...,x_n)}$

## Naive Bayes:

The approach presented above demands a large amount of information, in order to estimate the probability distribution for all different possible combinations of values.

Instead I will use the "Naive Bayes" approach: I will assume __independency__ between every pair of features (and will preprocces the data accordingly). The independency assumption gives $P(B) \cdot P(A|B)=P(B) \cdot P(A)$ , and therefore the calculated probabilty can be simplified to:
### $P(y|x) = \frac{P(y) \cdot \prod_{i=1}^{n} P(x_i|y)}{P(x_1, x_2,...,x_n)}$

We will need to determine what class gets the highest probability for each data point. Solely for the purpose of comparison, we can calculate just the numerator (because the denominator is constant):
### $P(y|x) \propto P(y) \cdot \prod_{i=1}^{n} P(x_i|y)$

- note: by calculating just the numerator we lower our precision in __predicting__ the probabilities of a data point being in a class. therefore, we refer mainly to the __comparison__ between classes (by getting the maximum value) rather than the actual probabilty value for the data point being in that class.
    
Another simplification is needed: calculating the above mentioned equation still requires claculating the conditional probabilities of $P(x_i|y)$. We can avoid doing so using __probability density function__ (PDF).)
(https://en.wikipedia.org/wiki/Probability_density_function, https://en.wikipedia.org/wiki/Bayes%27_theorem#:~:text=In%20principle%2C%20Bayes'%20theorem%20applies,relevant%20densities%20(see%20Derivation)
    For continuous random variables, the PDF is defined: $P[a \leq X \leq b] = \int_a^b {f(x)dx}$ where f is the PDF.
Probabilty for a specific event to occur will be $P(X=x_0) = F(x_0) - F(x_0) = 0$ (Newton-Leibniz formula). Instead of observing specific  (discrete) events, we will observe a small region of events $P(|X-x_0| < Δ(x))$.
For this purpose the PDF can be described as $pdf(|X-x_0| < Δ(x))= \frac {P(|X-x_0| < Δ(x))}{Δ(x)}$.

Let's refer to $P(|X-x_0|< Δ(x)$ as $P(X \sim x)$.

So, for continuous dataset:
### $P(Y\sim y|X\sim x) \propto P(Y=y) \cdot \prod_{i=1}^{n} P(X\sim x_i|Y\sim y)$


Therefore, an alternative option for writing out probabilities proportion equation is:
### $PDF(y|x)Δ(y) \propto P(y) \cdot \prod_{i=1}^{n} PDF(x_i|y)Δ(x)$

Dividing by $Δ(y), Δ(x_i)$ will keep the proportion (they are positive constants):
### $P(y|x) \propto PDF(y|x) \propto P(y) \cdot \prod_{i=1}^{n} PDF(x_i|y)$
Summing up, we can calculate the relations between the __densities__ instead of the actual conditional probablities to understand which class is most likely for a specific data point.
In order to use this method, we will need to know the distribution of the data. We can either assume a specific distribution, or try to approximate it.

## Kernel Density Estimation
Kernel density estimation (KDE) is a non-parametric method for estimating the probability density function of a given random variable. Here's a wonderfull interactive explanation: https://mathisonian.github.io/kde/.
This method takes independent samples which are identically distributed, $x_1, ... x_n$.
Intuitively, the KDE takes a window that calls bandwidth (similar to standard deviation) and runs it through the dataset.
The more datapoints included inside the bandwidth, means greater density and results in higher value for the KDE.
The KDE values are calculated by a weighted average of the distance between any point in the window, to a given point x, then divided by the bandwidth (hence the density). the weights come from the hyperparameter 'kernel', which is a non-negative function (https://en.wikipedia.org/wiki/Kernel_density_estimation#Definition). Mathematically:
### $\frac{1}{n\cdot h} \cdot \sum_{i=1}^{n} K(\frac{x-x_i}{h})$
where K is the kernel function and h is the bandwidth.

## Algorithm:
### First part: Naive Bayes
- __Assuming normal distribution__ for all the dataset.
- Calculate Priors using the relative frequency of each class in the dataset.
- Calculate the density function's values for each data point being in each class.
- Calculate score (which is the product of prior and PDF values) for each data point being in each class
- Among all of the calculated values, assign each of the data points to the class that has the highest score.

### Second part: Not so Naive Bayes
- Calculate Priors using the relative frequency of each class in the dataset.
- For each feature and each class, fit the KDE with the data (4 features * 3 classes = 12 instances).
- Calculate score (which is the product of prior and PDF values) for each data point being in each class
- Among all of the calculated values, assign each of the data points to the class that has the highest score.

### Third part: Best possible hyperparameters
- Perform gridsearch for the two KDE hyperparameters: kernel and bandwith.
- calculate the mean of cross-validation score for each hyperparameters pairing.
- Get the best hyperparameters for the dataset and create new estimator with them.
- Predict using the new estimator and compare the result to the Naive Bayes method.

### Getting the dataset ready

In [31]:
from sklearn.datasets import load_iris
import numpy as np
from scipy.stats import norm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [13]:
iris = load_iris()
data = np.c_[iris.data, iris.target]

In [32]:
X = data[:, :-1]
y = data[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [15]:
def splitByClass(dataset):
    split = {}
    for idx, class_num in enumerate(y):
        if class_num not in split:
            split[class_num] = dataset[idx]
        else:
            split[class_num] = np.vstack((split[class_num], dataset[idx]))
    return split

split = splitByClass(X)

# First part: Naive Bayes

In [16]:
def fitPredict(npDist, data, row):
    params = npDist.fit(data)
    arg = params[:-2]
    mean = params[-2]
    std = params[-1]
    pdf = npDist.pdf(X[row], loc=mean, scale=std, *arg)
    return pdf

### In the score calculation, instead of calculating the product of probabilities we'll calculate the sum of logs. That is to avoid getting very small numerical values, which can result in numerical instability. Taking a log of that product (very small positive number) results in a much larger negative value, thus avoiding the problem. Also, the log of product equals to the sum of logs so we can calculate the following score:

In [25]:
def score(prior, dist0, dist1, dist2, dist3):
    return np.log(prior) +  np.log(dist0) +  np.log(dist1) +  np.log(dist2) +  np.log(dist3)

### Calculate prior probabilities P(y)

In [18]:
prior_setosa = len(X[y==0]) / len(X)
prior_versicolor = len(X[y==1]) / len(X)
prior_virginica = len(X[y==2]) / len(X)

In [19]:
normdist = norm

In [20]:
def totalScores(dist, data, splitted):
    Y_pred = []
    for i in range(len(data)):
        d = {0: score(prior_setosa, fitPredict(dist, splitted[0][:, 0], i)[0],
            fitPredict(dist, splitted[0][:, 1], i)[1],
            fitPredict(dist, splitted[0][:, 2], i)[2],
            fitPredict(dist, splitted[0][:, 3], i)[3]),
             1: score(prior_versicolor, fitPredict(dist, splitted[1][:, 0], i)[0],
            fitPredict(dist, splitted[1][:, 1], i)[1],
            fitPredict(dist, splitted[1][:, 2], i)[2],
            fitPredict(dist, splitted[1][:, 3], i)[3]), 
              2: score(prior_virginica, fitPredict(dist, splitted[2][:, 0], i)[0],
            fitPredict(dist, splitted[2][:, 1], i)[1],
            fitPredict(dist, splitted[2][:, 2], i)[2],
            fitPredict(dist, splitted[2][:, 3], i)[3])}
        madataVal = max(d, key = lambda k: d[k])
        Y_pred.append(madataVal)
    return Y_pred

In [23]:
y_predNorm = totalScores(normdist, X, split)
norm_cm = confusion_matrix(y, y_predNorm)

In [26]:
norm_cm

array([[50,  0,  0],
       [ 0, 47,  3],
       [ 0,  3, 47]], dtype=int64)

# Second part: Not so Naive Bayes

In [51]:
from sklearn.base import BaseEstimator
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [38]:
class KDENaiveBayes(BaseEstimator):
    """A version of Naive Bayes that uses KDE to estimate the 
    density distribution  for every combination of feature and class,
    instead of assuming single distribution for all of the dataset"""
    
    def __init__(self, kernel = 'gaussian', bandwidth = 1):
        self.kernel = kernel
        self.bandwidth = bandwidth
        
    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y))
        self.splitByClass_ = [X[y == yi] for yi in self.classes_]
        self.KDEs_ = [KernelDensity(bandwidth=self.bandwidth,
                                   kernel=self.kernel).fit(x) for x in self.splitByClass_]
        return self
    
    def predict_proba(self, X):
        self.logPriors_ = [np.log(np.size(x[0], axis=0) / np.size(X[0], axis=0)) for x in self.splitByClass_]
        logScore = np.array([kde.score_samples(X) for kde in self.KDEs_]).T
        inverse = np.exp(logScore + self.logPriors_)
        return inverse / inverse.sum(1, keepdims=True)
        
    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), 1)]

In [39]:
kde = KDENaiveBayes()
kde.fit(X_train,y_train)
y_pred = kde.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[16,  0,  0],
       [ 0, 17,  1],
       [ 0,  2,  9]], dtype=int64)

In [40]:
cross_val_score(KDENaiveBayes(), X, y, scoring='accuracy').mean()

0.9266666666666667

### Let's quickly compare to scikit-learn's Naive Bayes performance

In [43]:
gnb = GaussianNB()
cross_val_score(GaussianNB(), X, y, scoring='accuracy').mean()

0.9533333333333334

### We're outscored :( Let's see if we can get a better result

# Third part: Best possible hyperparameters

### Now we'll add a method named 'GridFitParams' to our class, in order to find the best hyperparameters

In [46]:
class KDENaiveBayes(BaseEstimator):
    """A version of Naive Bayes that uses KDE to estimate the 
    density distribution  for every combination of feature and class,
    instead of assuming single distribution for all of the dataset"""
    
    def __init__(self, kernel = 'gaussian', bandwidth = 0.5):
        self.kernel = kernel
        self.bandwidth = bandwidth
        
    def GridFitParams(self, X, y):
        bandwidths = np.arange(start=0.05, stop=2.0, step=0.05)
        grid = GridSearchCV(KDENaiveBayes(), {'bandwidth': bandwidths,
                                      'kernel': ['gaussian', 'tophat', 'epanechnikov',
                                                 'exponential', 'linear', 'cosine']}, scoring='accuracy')
        grid.fit(X, y)
        print("best parameters: ", grid.best_params_)
        return grid.best_params_
    
    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y))
        self.splitByClass_ = [X[y == yi] for yi in self.classes_]
        self.KDEs_ = [KernelDensity(bandwidth=self.bandwidth,
                                   kernel=self.kernel).fit(x) for x in self.splitByClass_]
        return self
    
    def predict_proba(self, X):
        self.logPriors_ = [np.log(np.size(x[0], axis=0) / np.size(X[0], axis=0)) for x in self.splitByClass_]
        logScore = np.array([kde.score_samples(X) for kde in self.KDEs_]).T
        inverse = np.exp(logScore + self.logPriors_)
        return inverse / inverse.sum(1, keepdims=True)
        
    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), 1)]

In [52]:
fittedKDE = KDENaiveBayes()
best_params = fittedKDE.GridFitParams(X, y)
best_params

best parameters:  {'bandwidth': 1.05, 'kernel': 'tophat'}


{'bandwidth': 1.05, 'kernel': 'tophat'}

In [53]:
cross_val_score(KDENaiveBayes(kernel=best_params['kernel'], bandwidth=best_params['bandwidth']),
                X, y, scoring='accuracy').mean()

0.9600000000000002

### That's a win!